In [2]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from collections import defaultdict
from pprint import pprint

from config.assistant import *

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
#from config.site_config import *

In [2]:
# searchType=search
# searchword=컴퓨터비전         # 검색어
# loc_mcd=101000,102000         # 지역, 101000(서울),102000(경기)
# company_cd=0,1,2,3,4,5,6,7    # 채용제공구분 / 9(파견/대행), 10(헤드헌팅)
# search_optional_item=y
# search_done=y
# panel_count=y
# recruitPage=1                 # 현재 페이지뷰
# recruitSort=relation          # 정렬방법
# recruitPageCount=100          # 표시 개수
# inner_com_type=
# show_applied=                 # 지원한 공고 표시
# quick_apply=                  # 빠른 입사지원
# except_read=                  # 조회한 페이지 제외
# ai_head_hunting=
# mainSearch=n

In [3]:
search_words = ["컴퓨터비전", 'COMPUTER VISION',
                "영상처리", "IMAGE PROCESSING",
                "딥러닝", "DEEP LEARNING", 
                "머신러닝", "MACHINE LEARNING",
                "이미지인식", "IMAGE RECOGNITION",
                "이미지분석", "IMAGE ANALYSIS",
                "VISION AI",
                "OPENCV"
               ]

# search_words = ["이미지인식", "이미지분석"]
page_view_items = 100
company_cd='0,1,2,3,4,5,6,7'
base_link = 'https://www.saramin.co.kr'
job_group = {}


In [4]:

# 저장된 키워드 별 검색
for search_word in search_words:
    tic()
    item_count = 0
    current_page = 1
    
    # 검색 결과 리스트 페이지 별 확인
    search_loop = True
    while(search_loop):
        search_link = f'https://www.saramin.co.kr/zf_user/search/recruit'\
            + f'?search_area=main'\
            + f'&search_done=y'\
            + f'&search_optional_item=n'\
            + f'&searchType=search'\
            + f'&recruitSort=relation'\
            + f'&searchword={search_word}'\
            + f'&recruitPage={current_page}'\
            + f'&recruitPageCount={page_view_items}'\
            + f'&company_cd={company_cd}'\
            + f'&mainSearch=y'

        response = requests.get(search_link, verify=False)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        if len(soup.find_all('div', attrs={'class': 'info_no_result'})) == 1:
            search_loop = False
        else:
            current_page += 1

            page_items = len(soup.find_all('h2', attrs={'class': 'job_tit'}))

            # 해당 페이지 리스트 (1 ~ 100)
            for i in range(1, page_items+1):
                elements = soup.select(f'div.content > div:nth-child({i})')[0]

                # 채용공고명
                str_title = elements.find('a', attrs={'class': 'data_layer'})['title']

                # 회사명
                try:
                    str_company1 = elements.find('a', attrs={'class': 'track_event data_layer'}).text
                    #str_company = str_company.replace(' ', '').replace('\n', '').replace('(주)', '')
                    str_company2 = str_company1.replace(' ', '')
                    str_company3 = str_company2.replace('\n', '')
                    str_company = str_company3.replace('(주)', '')
                except:
                    print(str_company1)
                    print(str_company2)
                    print(str_company3)
                    print(str_company)

                # 채용공고 링크
                str_title_link = base_link + elements.find('a', attrs={'class': 'data_layer'})['href']

                # 회사 링크
                str_company_link = base_link + elements.find('a', attrs={'class': 'track_event data_layer'})['href']

                # Dictionary에 저장
                if str_company in job_group:    # 회사명 - 기존에 있으면
                    for company in job_group[str_company]:
                        if company['title'] == str_title:    # 채용공고명 - 기존에 있으면
                            break
                    else:
                        job_group[str_company].append({'title': str_title, \
                                                       'title_link': str_title_link, \
                                                       'company_link': str_company_link \
                                                       'input_date': datetime.date.today().isoformat()\
                                                      })

                else:    # 회사명 - 신규진입
                    job_group[str_company] = []
                    job_group[str_company].append({'title': str_title, \
                                                   'title_link': str_title_link, \
                                                   'company_link': str_company_link \
                                                   'input_date': datetime.date.today().isoformat()\
                                                  })

                item_count += 1
                
    print(f'{search_word}: {item_count}')
    toc()

컴퓨터비전: 1120
Elapsed time is 95.84467911720276 seconds.
COMPUTER VISION: 964
Elapsed time is 81.77384281158447 seconds.
영상처리: 1037
Elapsed time is 91.63178586959839 seconds.
IMAGE PROCESSING: 295
Elapsed time is 26.112196683883667 seconds.
딥러닝: 1158
Elapsed time is 100.3155210018158 seconds.
DEEP LEARNING: 1158
Elapsed time is 99.07968306541443 seconds.
머신러닝: 1450
Elapsed time is 122.66391611099243 seconds.
MACHINE LEARNING: 1445
Elapsed time is 123.3249671459198 seconds.
이미지인식: 222
Elapsed time is 20.000678777694702 seconds.
IMAGE RECOGNITION: 12
Elapsed time is 2.2892861366271973 seconds.
이미지분석: 714
Elapsed time is 62.77639722824097 seconds.
IMAGE ANALYSIS: 21
Elapsed time is 2.4867708683013916 seconds.
VISION AI: 657
Elapsed time is 55.909586906433105 seconds.
OPENCV: 232
Elapsed time is 19.707900047302246 seconds.


In [5]:
len(job_group)

2259

In [ ]:
머신러닝: 1450
MACHINE LEARNING: 1445
딥러닝: 1158
DEEP LEARNING: 1158
컴퓨터비전: 1120
영상처리: 1037
COMPUTER VISION: 964
이미지분석: 714
VISION AI: 657
IMAGE PROCESSING: 295
OPENCV: 232
이미지인식: 222
IMAGE ANALYSIS: 21
IMAGE RECOGNITION: 12

In [ ]:
9365
job_group 2259
